In [30]:
%reload_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/

"rm" no se reconoce como un comando interno o externo,
programa o archivo por lotes ejecutable.


In [72]:
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp
import numpy as np
import pandas as pd
from keras.utils import to_categorical
from keras.losses import CategoricalCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split as split

In [38]:
scaler = StandardScaler()

### Conjunto de datos: SDSS DR17

In [39]:
data = pd.read_csv('./SDSS/star_classification.csv')
cols = ['alpha','delta','u','g','r','i','z','redshift','class']
data = data[cols]
data["class"]=[0 if i == "GALAXY" else 1 if i == "STAR" else 2 for i in data["class"]]
print(data.head())
data = data.to_numpy()

        alpha      delta         u         g         r         i         z  \
0  135.689107  32.494632  23.87882  22.27530  20.39501  19.16573  18.79371   
1  144.826101  31.274185  24.77759  22.83188  22.58444  21.16812  21.61427   
2  142.188790  35.582444  25.26307  22.66389  20.60976  19.34857  18.94827   
3  338.741038  -0.402828  22.13682  23.77656  21.61162  20.50454  19.25010   
4  345.282593  21.183866  19.43718  17.58028  16.49747  15.97711  15.54461   

   redshift  class  
0  0.634794      0  
1  0.779136      0  
2  0.644195      0  
3  0.932346      0  
4  0.116123      0  


In [40]:
def prepare_dataset(data):
    X, Y = np.empty((0)), np.empty((0))
    X = data[:, :8]
    Y = data[:, 8]
    Y = to_categorical(Y, num_classes=3)
    return X, Y

In [42]:
X, Y = prepare_dataset(data)
X_train, X_test, Y_train, Y_test = split(X, Y, test_size = 0.3, random_state = 0)
X_train = scaler.fit_transform(X_train)
X_test  = scaler.transform(X_test)

lenx, input_shape = np.shape(X)

### Hiperparámetros del modelo

In [84]:
HP_LAYERS = hp.HParam('layers', hp.Discrete([8, 16]))
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([4,8,16,32]))
HP_LEARNING  = hp.HParam('learning_rate', hp.Discrete([4,8,16,32]))
# HP_BATCHSIZE = hp.HParam('batch_size', hp.Discrete([16, 32]))

callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_categorical_accuracy', mode='max',
                                   min_delta=0,
                                   patience=5,
                                   restore_best_weights=True)]
batch_size = 32
epochs = 1

In [85]:
# METRIC_ACCURACY = 'accuracy'
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
# with tf.summary.FileWriter('logs/hparam_tuning', sess.graph):
#     init = tf.initialize_all_variables()
#     sess.run(init)
    hp.hparams_config(
        hparams=[HP_LAYERS, HP_NUM_UNITS, HP_LEARNING],
        metrics=[hp.Metric('loss', display_name="Loss")])

In [86]:
def train_test_model(hparams):
    # Train LSTM model and predict on validation set
    model = keras.Sequential()
    model.add(Input(shape=(int(X_train.shape[1]),)))
    model.add(Dense(hparams[HP_NUM_UNITS], input_shape=(int(X_train.shape[1]),)))
    
    for i in range(hparams[HP_LAYERS]):        
        model.add(Dense(hparams[HP_NUM_UNITS], activation='relu'))
    model.add(Dense(3, activation=tf.nn.softmax))
     
    optimizer = keras.optimizers.Adam(learning_rate=hparams[HP_LEARNING]*10**(-4), beta_1=0.9, beta_2=0.999, epsilon=1e-3)
    model.compile(
            optimizer=optimizer,
            loss=CategoricalCrossentropy(),
            metrics=["categorical_accuracy"]
          )
    # Run with 1 epoch to speed things up for demo purposes
    # model.fit(x_train, y_train, epochs=1, callbacks=callbacks) 
    

    model.fit(X_train, Y_train, epochs=epochs, validation_data=(X_test, Y_test),
              callbacks=callbacks, batch_size=batch_size, shuffle=True)
#               , verbose=0)

    _, loss = model.evaluate(X_test, Y_test)


    return loss
    

In [87]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        loss = train_test_model(hparams)
        tf.summary.scalar("loss", loss, step=1)

In [88]:
session_num = 0

for deep_layers in HP_LAYERS.domain.values:
    for num_units in HP_NUM_UNITS.domain.values:
        for learning_rate in HP_LEARNING.domain.values:
#             for batch_size in HP_BATCHSIZE.domain.values:
            hparams = {

                HP_LAYERS: deep_layers,
                HP_NUM_UNITS: num_units,
                HP_LEARNING: learning_rate,
#                     HP_BATCHSIZE: batch_size
            }
            run_name = "run-%d" % session_num
            print('\n--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

print(session_num)


--- Starting trial: run-0
{'layers': 8, 'num_units': 4, 'learning_rate': 4}
Train on 70000 samples, validate on 30000 samples
30000/30000 [==============================] - 3s 115us/sample - loss: 0.7982 - categorical_accuracy: 0.7347 - loss: 0.8062 - categorical_ac

--- Starting trial: run-1
{'layers': 8, 'num_units': 4, 'learning_rate': 8}
Train on 70000 samples, validate on 30000 samples
30000/30000 [==============================] - 4s 124us/sample - loss: 0.5234 - categorical_accuracy: 0.7996

--- Starting trial: run-2
{'layers': 8, 'num_units': 4, 'learning_rate': 16}
Train on 70000 samples, validate on 30000 samples
30000/30000 [==============================] - 5s 154us/sample - loss: 0.5583 - categorical_accuracy: 0.7535

--- Starting trial: run-3
{'layers': 8, 'num_units': 4, 'learning_rate': 32}
Train on 70000 samples, validate on 30000 samples
30000/30000 [==============================] - 5s 153us/sample - loss: 0.4006 - categorical_accuracy: 0.9606

--- Starting trial: r

Train on 70000 samples, validate on 30000 samples
30000/30000 [==============================] - 4s 143us/sample - loss: 0.3646 - categorical_accuracy: 0.9212

--- Starting trial: run-21
{'layers': 16, 'num_units': 8, 'learning_rate': 8}
Train on 70000 samples, validate on 30000 samples
30000/30000 [==============================] - 4s 138us/sample - loss: 1.6887 - categorical_accuracy: 0.9597

--- Starting trial: run-22
{'layers': 16, 'num_units': 8, 'learning_rate': 16}
Train on 70000 samples, validate on 30000 samples
30000/30000 [==============================] - 4s 135us/sample - loss: 0.2494 - categorical_accuracy: 0.9112

--- Starting trial: run-23
{'layers': 16, 'num_units': 8, 'learning_rate': 32}
Train on 70000 samples, validate on 30000 samples
30000/30000 [==============================] - 5s 173us/sample - loss: 0.2092 - categorical_accuracy: 0.9400 - loss: 0.2088 - categorical_accuracy: 0.94

--- Starting trial: run-24
{'layers': 16, 'num_units': 16, 'learning_rate': 4}
T

In [89]:
rm -rf /tmp/tb_logs/

SyntaxError: invalid syntax (<ipython-input-89-8a7b594d7468>, line 1)

In [13]:
# !kill 16011

In [ ]:
%tensorboard --logdir logs/hparam_tuning

Launching TensorBoard...